In [1]:
import re
import time

import numpy as np
import pandas as pd
import requests
from progressbar import progressbar

## Scraping

### Scraping Functions

In [2]:
# function to scrape reddit page (takes a reddit .json url)
# returns posts 

headers = {'User-Agent' : 'override this bad boy!'}

def scraper_bike(url):
    posts = []
    after = {}

    for page in progressbar(range(40)):
        params = {'after' : after}
        url = url
        pagepull = requests.get(url = url, params = params, headers = headers)
        page_dict = pagepull.json()
        posts.extend(page_dict['data']['children'])
        after = page_dict['data']['after']
        time.sleep(.2)
        
    return posts

In [3]:
# function to convert posts to DataFrame - won't allow duplicate posts since unique id 'name' is set as index
# Extract: name (as index) and subreddit, selftext, title (as columns)

def posts_to_df(post_list):
    i = 0
    post_dict = {}
    
    for post in post_list:
        ind = post_list[i]['data']
        post_dict[ind['name']] = [ind['subreddit'], ind['title'], ind['selftext']]
        i += 1

    df_name = pd.DataFrame(post_dict)
    df_name = df_name.T
    df_name.columns = ['subreddit', 'title', 'selftext'] #'selftext'
    
    return df_name

In [4]:
# takes scraper function and url - outputs dataframe

def scrape_to_df(scrape_func, url):
    
    return posts_to_df(scrape_func(url))

In [5]:
#### If you want to scrape repeatedly over time and add to a csv
# scrape, import csv, concat, drop duplicate, and output to csv
# takes in scraper function, url, csv filename to import, csv filename to output
# Outputs - Concatenated DataFrame as csv

def scrape_add(scrape_func, url, import_file, export_file):
    scrape_df = posts_to_df(scrape_func(url))
    imported_df = pd.read_csv(import_file, index_col = 'Unnamed: 0')
    concat_df = pd.concat([imported_df, scrape_df])
    concat_df = concat_df[~concat_df.index.duplicated(keep='first')]
    concat_df.to_csv(export_file)

### Run Scrape

In [9]:
# Run this and comment out pd.read_csv lines in data cleaning / preprocessing to use freshly scraped data
# You can also put in any 2 subreddits in as the URL and get results for those

nfltest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nfl.json')
nbatest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nba.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:34 Time:  0:00:34
100% (40 of 40) |########################| Elapsed Time: 0:00:24 Time:  0:00:24


In [10]:
politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:24 Time:  0:00:24
100% (40 of 40) |########################| Elapsed Time: 0:00:25 Time:  0:00:25


In [ ]:
nbatest.shape

In [ ]:
nfltest.shape

##### These scrape_add functions add to already built csvs

In [11]:
scrape_add(scraper_bike, 'https://www.reddit.com/r/CollegeBasketball/new.json', 'NCAA_Posts_Update2.csv', 'NCAA_Posts_Update3.csv')
scrape_add(scraper_bike, 'https://www.reddit.com/r/AskScience/new.json', 'AskSci_Posts_Update2.csv', 'AskSci_Posts_Update3.csv')
scrape_add(scraper_bike, 'https://www.reddit.com/r/nba/new.json', 'NBA_Posts_Update2.csv', 'NBA_Posts_Update3.csv')
scrape_add(scraper_bike, 'https://www.reddit.com/r/nfl/new.json', 'NFL_Posts_Update2.csv', 'NFL_Posts_Update3.csv')

100% (40 of 40) |########################| Elapsed Time: 0:00:29 Time:  0:00:29


FileNotFoundError: [Errno 2] File b'NCAA_Posts_Update2.csv' does not exist: b'NCAA_Posts_Update2.csv'

### Data Cleaning / Preprocessing

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('max_colwidth', 300)

In [3]:
# drop column

nfltest = nfltest.drop(columns = 'selftext')
nbatest = nbatest.drop(columns = 'selftext')

NameError: name 'nfltest' is not defined

In [14]:
# merge subreddit data

train = pd.concat([nfltest, nbatest])

In [15]:
train

,subreddit,title
t3_dcc7g6,nfl,Water Cooler Wednesday
t3_dbxfyg,nfl,Official Week 4 /r/NFL Power Rankings
t3_dcajbu,nfl,Tom Brady has been wearing the same shoulder pads since his freshman year at Michigan in 1995. They're older than 5 of his current team-mates.
t3_dca6d5,nfl,Percy Harvin Says He Was High Every Game He Played
t3_dc44f6,nfl,[Jaguars] The Jaguars are giving out a bandana and a mustache to any fan who purchases tickets to the team's 2 home games this month
t3_dc9s8i,nfl,"[Cox] Patriots linebacker Kyle Van Noy is the AFC's Defensive Player of the Week. Eight tackles, two sacks, two forced fumbles against the Bills, plus the pressure that produced Jamie Collins' game-sealing INT. He's been excellent this season."
t3_dc9lv2,nfl,Jameis Winston named NFC offensive player of the week
t3_dcb1s4,nfl,Andy Reid once demolished a 40-oz steak in 19 minutes
t3_dc12hm,nfl,"[Graham] On QBs who refuse to slide, Brett Favre says: ""It can become toxic that every time you run the ball you want to steamroll someone because the fans love it, and two years later you’re beat up and out of football. Now the fans are loving the next guy.”"
t3_dby4je,nfl,"The Jaguars are 2-2. They are 1-1 at home and 1-1 on the road. They have scored 84 points and have allowed 84 points. They are averaging 33.1 yards per drive, and are giving up an average of 33.1 yards per drive"


##### Tokenize (grab only word characters)

In [16]:
word_tokenizer = RegexpTokenizer(r'\w+')

In [17]:
train['title'] = train['title'].map(lambda x: word_tokenizer.tokenize(x.lower()))

In [18]:
# rejoin list of tokenized words into single string for each row

train['title'] = train['title'].map(lambda x: ' '.join(x))

In [19]:
train['title'][0:5]

t3_dcc7g6                                                                                                                          water cooler wednesday
t3_dbxfyg                                                                                                            official week 4 r nfl power rankings
t3_dcajbu    tom brady has been wearing the same shoulder pads since his freshman year at michigan in 1995 they re older than 5 of his current team mates
t3_dca6d5                                                                                              percy harvin says he was high every game he played
t3_dc44f6              jaguars the jaguars are giving out a bandana and a mustache to any fan who purchases tickets to the team s 2 home games this month
Name: title, dtype: object

In [20]:
train['title'][0:5]

t3_dcc7g6                                                                                                                          water cooler wednesday
t3_dbxfyg                                                                                                            official week 4 r nfl power rankings
t3_dcajbu    tom brady has been wearing the same shoulder pads since his freshman year at michigan in 1995 they re older than 5 of his current team mates
t3_dca6d5                                                                                              percy harvin says he was high every game he played
t3_dc44f6              jaguars the jaguars are giving out a bandana and a mustache to any fan who purchases tickets to the team s 2 home games this month
Name: title, dtype: object

### Train test split and converting series to list of strings then to array

In [21]:
X = train[['title']]
y = train['subreddit']

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.25,
                                                    random_state=42,
                                                    stratify=y)

In [23]:
# baseline is

y.value_counts(normalize=True)

nfl    0.560216
nba    0.439784
Name: subreddit, dtype: float64

In [24]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)

In [25]:
len(clean_train_data)

1251

In [26]:
clean_train_data[0:2]

['mvp for kawhi',
 'gardner minshew s current career completion percentage is 69 420']

In [27]:
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [28]:
len(clean_test_data)

418

### Count Vectorizer

In [29]:
stopwords = set(stopwords.words('english'))

extra_stopwords = ['nba', 'basketball', 'football', 'nfl']

stopwords.update(extra_stopwords)

In [30]:
# instantiate our CountVectorizer. This counts the number of appearances of all the words in our training data and
# eliminates common english stop words. 5000 max features works well for our purposes (tested various numbers). Our
# data is already preprocessed and tokenized manually earlier. ngram_range is 1,3, although all or nearly all our
# features are single words

vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words=stopwords,
                             max_features=5000,
                             ngram_range=(1, 3))

In [31]:
# fit our training data and test data lists to our count_vectorizer

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

In [32]:
# convert to array

train_data_features = train_data_features.toarray()

In [33]:
# check shapes

train_data_features.shape, test_data_features.shape

((1251, 5000), (418, 5000))

In [34]:
# I wanted check that the features corpus was as expected - removed print statement for readability

vocab = vectorizer.get_feature_names()

In [35]:
vocab[0:1000]

['000',
 '04',
 '10',
 '10 catches',
 '10 catches 223',
 '10 points',
 '10 yards',
 '100',
 '100 greatest',
 '100 greatest games',
 '1000',
 '101',
 '101 beyond',
 '101 beyond assists',
 '106',
 '11',
 '12',
 '13',
 '13 weeks',
 '14',
 '14 17',
 '140',
 '1400',
 '15',
 '150',
 '150 yard',
 '151',
 '16',
 '16 17',
 '16 games',
 '16 tds',
 '16m',
 '17',
 '18',
 '184',
 '19',
 '19 season',
 '1950',
 '1986',
 '1990',
 '1999',
 '1st',
 '1st 4th',
 '1st amp',
 '1st amp 2nd',
 '1st place',
 '20',
 '200',
 '200 games',
 '2000',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2010s',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2019 20',
 '2019 media',
 '2019 media day',
 '2019 season',
 '2020',
 '2020 olympics',
 '21',
 '22',
 '223',
 '223 yards',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '29 carries',
 '2nd',
 '2nd round',
 '2nd time',
 '30',
 '30 franchises',
 '30 franchises performance',
 '30 years',
 '300',
 '300 yard',
 '31',
 '32',
 '32 games'

## MODELING

### Logistic Regression

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
# fit logistic regression model

lr = LogisticRegression(penalty='l2')

In [38]:
# shape check

train_data_features.shape, y_train.shape

((1251, 5000), (1251,))

In [39]:
lr.fit(train_data_features, y_train)

C:\Users\Rocio\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
lr.score(train_data_features, y_train)

0.991207034372502

In [41]:
lr.score(test_data_features, y_test)

0.8851674641148325

### Feature comparison

Creates a dataframe that matches features to coefficients

In [42]:
coef_list = lr.coef_.tolist()

In [43]:
coef_list = coef_list[0]

In [44]:
coef_df = pd.DataFrame({'features': vectorizer.get_feature_names(),
                        'coefs': coef_list})

In [45]:
coef_df.sort_values(by = ['coefs'])

,features,coefs
1492,kawhi,-1.275988
1612,lebron,-1.255621
1556,lakers,-1.144756
977,finals,-0.998200
1228,harden,-0.963576
1134,getting,-0.899776
1495,kd,-0.859965
1249,height,-0.839772
1527,knicks,-0.826608
1364,india,-0.825835


### Let's throw out these unfair words and rerun

In [ ]:
stopwords = set(stopwords.words('english'))

extra_stopwords = ['nba', 'basketball', 'football', 'nfl']

stopwords.update(extra_stopwords)

In [46]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = stopwords,
                             max_features = 5000,
                             ngram_range = (1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1251, 5000), (418, 5000))

In [47]:
lr.fit(train_data_features, y_train)

C:\Users\Rocio\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
lr.score(train_data_features, y_train)

0.991207034372502

In [49]:
lr.score(test_data_features, y_test)

0.8851674641148325

In [50]:
coef_list = lr.coef_.tolist()
coef_list = coef_list[0]

coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
1492,kawhi,-1.275988
1612,lebron,-1.255621
1556,lakers,-1.144756
977,finals,-0.998200
1228,harden,-0.963576
1134,getting,-0.899776
1495,kd,-0.859965
1249,height,-0.839772
1527,knicks,-0.826608
1364,india,-0.825835


### Decision Tree

In [51]:
from sklearn.tree import DecisionTreeClassifier

In [52]:
tree = DecisionTreeClassifier()

In [53]:
tree.fit(train_data_features, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [54]:
tree.score(train_data_features, y_train)

0.9992006394884093

In [55]:
tree.score(test_data_features, y_test)

0.8157894736842105

### Random Forest

In [56]:
from sklearn.ensemble import RandomForestClassifier

In [57]:
forest = RandomForestClassifier(n_estimators = 100)

In [58]:
forest.fit(train_data_features, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [59]:
forest.score(train_data_features, y_train)

0.9992006394884093

In [60]:
forest.score(test_data_features, y_test)

0.8516746411483254

###  Matrix on Logistic Regression

In [61]:
from sklearn.metrics import confusion_matrix

In [62]:
y_pred = lr.predict(test_data_features)

In [63]:
cm = confusion_matrix(y_test, y_pred)

In [64]:
cm_df = pd.DataFrame(cm,
                    columns=['predict_neg', 'predict_pos'],
                    index = ['actual_neg', 'actual_pos'])

In [65]:
cm_df

,predict_neg,predict_pos
actual_neg,171,13
actual_pos,35,199


## Checking where our model failed

In [66]:
comparison_df = pd.DataFrame({'y_actual' : y_test,
             'y_predicted' : y_pred})

In [67]:
mismatch_df = comparison_df[comparison_df['y_actual'] != comparison_df['y_predicted']]

In [68]:
mismatch2_df = pd.concat([mismatch_df, X_test], axis = 1)

C:\Users\Rocio\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [69]:
# All incorrect predictions with titles

mismatches = mismatch2_df.dropna()

In [70]:
mismatches

,y_actual,y_predicted,title
t3_daagf7,nba,nfl,westbrook i don t have to have the ball to impact the game i don t have to score i don t have to do anything i can defend i can rebound i can pass i can lead
t3_dairh3,nba,nfl,patrick mccaw about to get a ring this year as well
t3_dasks0,nba,nfl,oc most exciting and boring games of the 2018 2019 nba season
t3_davnev,nfl,nba,why doesn t the nfl have a special teams player of the year award
t3_dax3xh,nba,nfl,will the pistons look to trade drummond or griffin this season
t3_day81i,nba,nfl,ray allen went 8 for 11 from the three point line in game 2 of the 2010 finals setting a finals record he went 4 for 30 from three in the other games of the series
t3_dazc5e,nba,nfl,compared the nfl regular season the nba reg season feels more like one long pre season what can be done to change that
t3_db26uh,nba,nfl,ray allen went 8 13 from 3 after 2 games in the 2010 finals before ron artest gave him a dirty knee to the thigh at the beginning of game 3 he went 4 28 from 3 after the knee from artest
t3_db3asm,nfl,nba,shaq barrett 2019 stats 20 tkls 9 sacks 3 ff 1 int
t3_db5c6q,nfl,nba,taysom hill trucks xavier woods on snf


### Let's try TF-IDF

Term Frequency / Inverse Document Frequency

TF(w) = (Number of times term w appears in a document) / (Total number of terms in the document)

IDF(w) = log_e(Total number of documents / Number of documents with term w in it)

In [71]:
tfidf_vec = TfidfVectorizer(analyzer="word",
                            tokenizer=None,
                            preprocessor=None,
                            stop_words=stopwords,
                            max_features=5000,
                            ngram_range=(1, 3))

In [72]:
train_data_features = tfidf_vec.fit_transform(clean_train_data)

test_data_features = tfidf_vec.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1251, 5000), (418, 5000))

In [73]:
lr.fit(train_data_features, y_train)

C:\Users\Rocio\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [74]:
lr.score(train_data_features, y_train)

0.9800159872102319

In [75]:
lr.score(test_data_features, y_test)

0.8732057416267942

### Let's try on some other subreddits

In [76]:
train = pd.concat([politics_test, conservative_test])

In [77]:
X = train[['title']]
y = train['subreddit']

In [ ]:
# politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
# conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [79]:
politics_test = politics_test.drop(columns = 'selftext')
conservative_test = conservative_test.drop(columns = 'selftext')

train = pd.concat([politics_test, conservative_test])
tokenizer = RegexpTokenizer(r'\w+')

train['title'] = train['title'].map(lambda x: tokenizer.tokenize(x.lower()))
train['title'] = train['title'].map(lambda x: ' '.join(x))

In [80]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)
    
    
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [81]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             max_features = 5000,
                             ngram_range = (1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

vocab = vectorizer.get_feature_names()

### Modeling

In [82]:
lr = LogisticRegression(penalty = 'l2')

In [83]:
train_data_features.shape, y_train.shape

((1387, 5000), (1387,))

In [84]:
lr.fit(train_data_features, y_train)

lr.score(train_data_features, y_train)

C:\Users\Rocio\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9834174477289113

In [85]:
lr.score(test_data_features, y_test)

0.7688984881209503

In [86]:
coef_list = lr.coef_.tolist()

coef_list = coef_list[0]

In [87]:
coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
663,dems,-1.303761
625,deduction,-1.123040
266,biden,-1.122455
1189,illegal,-0.935964
91,admits,-0.883802
167,aoc,-0.856340
657,democrats subpoena,-0.853423
1232,individual,-0.852963
1121,hillary,-0.846194
1017,going,-0.838769


In [89]:
tree = DecisionTreeClassifier()

In [90]:
tree.fit(train_data_features, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [91]:
tree.score(train_data_features, y_train)
tree.score(test_data_features, y_test)

0.7235421166306696

In [92]:
forest = RandomForestClassifier(n_estimators = 100)
forest.fit(train_data_features, y_train)



0.7667386609071274

In [93]:
forest.score(train_data_features, y_train)

0.9978370583994232

In [94]:
forest.score(test_data_features, y_test)

0.7667386609071274

In [95]:
y_pred = lr.predict(test_data_features)

In [96]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm,
                    columns=['predict_neg', 'predict_pos'],
                    index = ['actual_neg', 'actual_pos'])

In [97]:
cm_df

,predict_neg,predict_pos
actual_neg,178,55
actual_pos,52,178


In [98]:
comparison_df = pd.DataFrame({'y_actual' : y_test,
             'y_predicted' : y_pred})

In [99]:
mismatch_df = comparison_df[comparison_df['y_actual'] != comparison_df['y_predicted']]

In [100]:
mismatch2_df = pd.concat([mismatch_df, X_test], axis = 1)

C:\Users\Rocio\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [101]:
mismatches = mismatch2_df.dropna()

In [102]:
mismatches

,y_actual,y_predicted,title
t3_d95rae,Conservative,politics,New poll: Majority of Americans against impeachment and removal of Trump
t3_d98sy8,Conservative,politics,GOP Raises Nearly $1 Million on Day of Impeachment Announcement
t3_d9abwn,Conservative,politics,Ukraine Pres. on Trump phone call: 'Nobody pushed me'
t3_d9hy80,Conservative,politics,"Romney Defends Pelosi Impeachment Inquiry, Attacks Trump"
t3_d9l1wu,Conservative,politics,"Transcript Reveals Trump, Ukrainian President Argued For Fifteen Minutes About Who Would Hang Up First"
t3_d9mrav,Conservative,politics,"Florida Middle School Lesson Labels Trump as ""Idiot"""
t3_d9oic0,Conservative,politics,"He’s 67 points down in N.H., but Bill Weld says his quixotic campaign against Trump is on track"
t3_d9rjxa,Conservative,politics,House to Take 2-Week ‘Recess’ Vacation Following ‘Impeachment Inquiry’
t3_d9sibo,Conservative,politics,New Warren poster x-posted for all
t3_d9t6tk,Conservative,politics,The speech they're trying to hide: President Trump's stellar UN speech
